# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1603466875534_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [9]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://fall2020-cse6242"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
trips.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)

In [19]:
mtrips.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: string (nullable = true)
 |-- pcount: double (nullable = true)

In [20]:
wp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PULocationID: string (nullable = true)
 |-- weighted_profit: double (nullable = true)

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [10]:
def user():
    # Returns a string consisting of your GT username.
    return 'helfayoumy3'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips longer than 2 miles.

In [11]:
def long_trips(trips):
    trips = trips.where(col("trip_distance")  >=  2.0)    
    return trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [12]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    mtrips = trips.join(lookup, col("DOLocationID") == col("LocationID")).where(col("Borough") == "Manhattan")\
        .groupby(col("DOLocationID")).agg({'passenger_count':'sum'})\
        .withColumnRenamed("sum(passenger_count)", "pcount")\
        .orderBy(col("pcount").desc()).limit(20)
    return mtrips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [13]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    wp1 = trips.groupBy(col("PULocationID")).agg({"total_amount":"mean", "PULocationID":"count"})
    wp2 = trips.join(mtrips, ["DOLocationID"])
    wp2 = wp2.groupBy(col("PULocationID")).agg({"DOLocationID" : "count"})
    wp3 = wp1.join(wp2, "PULocationID")
    wp = wp3.withColumn("weighted_profit", col("avg(total_amount)") * (col("count(DOLocationID)") / col("count(PULocationID)")))\
    .select("PULocationID", "weighted_profit").orderBy(col("weighted_profit").desc()).limit(20)
    return wp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [14]:
def final_output(wp, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    final = lookup.join(wp, lookup["LocationID"] == wp["PULocationID"])\
    .select("Zone", "Borough", "weighted_profit")\
    .orderBy(col("weighted_profit").desc()).limit(20)

    return final

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the address to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear ina folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [17]:
bucket = 's3://cse6242-helfayoumy3/output-small7'
#main('small',bucket)
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

helfayoumy3
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|29.360455779130838|
|Flushing Meadows-...|       Queens| 27.30484573361766|
|       South Jamaica|       Queens|26.294916239873476|
|     Randalls Island|    Manhattan| 24.15098994022752|
|        Astoria Park|       Queens| 21.70641711214752|
|Briarwood/Jamaica...|       Queens|19.945064631789332|
|Springfield Garde...|       Queens|19.468309288781903|
|             Jamaica|       Queens|19.283943000137885|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808373006|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens|17.005450640079548|
|Eltingville/Annad...|Staten Island| 16.83776475694444|
|         JFK Airport|       Queens|16.777725348249643|
|        Batt

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [16]:
print(user())
trips, lookup = load_data(size='large')
trips = long_trips(trips)
mtrips = manhattan_trips(trips, lookup)
wp = weighted_profit(trips, mtrips)
final = final_output(wp,lookup)

# Outputs the results for you to visually see
final.show()

#trips.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

helfayoumy3
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens| 29.36045577913084|
|Flushing Meadows-...|       Queens|27.304845733617682|
|       South Jamaica|       Queens|26.294916239873473|
|     Randalls Island|    Manhattan| 24.15098994022752|
|        Astoria Park|       Queens|21.706417112147527|
|Briarwood/Jamaica...|       Queens| 19.94506463178933|
|Springfield Garde...|       Queens|19.468309288781906|
|             Jamaica|       Queens|19.283943000137903|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808373003|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens|17.005450640079545|
|Eltingville/Annad...|Staten Island| 16.83776475694445|
|         JFK Airport|       Queens|16.777725348249636|
|        Batt